In [4]:
!pip install qiskit
!pip install qiskit-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 17.5 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.29.241-py3-none-any.whl size=685423 sha256=d14eb9257e2e8c59

In [7]:
!pip install qiskit qiskit-algorithms qiskit-optimization pygame
!apt-get install -y xvfb
!pip install pyvirtualdisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 600))
display.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 7,815 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [10]:
import pygame
import math
import random
import time
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA  # Updated import
from qiskit_algorithms.optimizers import COBYLA  # Add a classical optimizer
from qiskit.primitives import Sampler  # Use the updated Sampler


In [13]:
# Node class representing each point in the tree
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.cost = 0.0

class RRTStar:
    def __init__(self, start, goal, obstacle_list, x_range, y_range, step_size=20, max_iter=500, radius=30):
        self.start = Node(*start)
        self.goal = Node(*goal)
        self.obstacle_list = obstacle_list
        self.x_range = x_range
        self.y_range = y_range
        self.step_size = step_size
        self.max_iter = max_iter
        self.radius = radius
        self.nodes = [self.start]
        pygame.init()
        self.screen = pygame.display.set_mode((x_range[1], y_range[1]))
        pygame.display.set_caption("RRT* Pathfinding Visualization")
        self.clock = pygame.time.Clock()
        self.screen.fill((255, 255, 255))

    def draw_circle(self, x, y, radius, color):
        pygame.draw.circle(self.screen, color, (int(x), int(y)), int(radius))

    def draw_line(self, x1, y1, x2, y2, color):
        pygame.draw.line(self.screen, color, (int(x1), int(y1)), (int(x2), int(y2)))

    def distance(self, node1, node2):
        return math.sqrt((node1.x - node2.x) ** 2 + (node1.y - node2.y) ** 2)

    def get_random_node(self):
        x = random.uniform(self.x_range[0], self.x_range[1])
        y = random.uniform(self.y_range[0], self.y_range[1])
        return Node(x, y)

    def nearest_node(self, random_node):
        return min(self.nodes, key=lambda node: self.distance(node, random_node))

    def is_collision_free(self, node1, node2):
        for (ox, oy, size) in self.obstacle_list:
            steps = int(max(abs(node2.x - node1.x), abs(node2.y - node1.y)))
            for i in range(steps):
                x = node1.x + (node2.x - node1.x) * i / steps
                y = node1.y + (node2.y - node1.y) * i / steps
                if math.hypot(ox - x, oy - y) <= size:
                    return False
        return True

    def quantum_optimization(self, candidate_nodes):
        # Create a Quadratic Program to optimize node selection
        qp = QuadraticProgram(name="Node Selection")
        for i in range(len(candidate_nodes)):
            qp.binary_var(name=f'x{i}')  # Binary decision variable for each node

        # Objective: Minimize the total cost (distance to goal + cost to reach the node)
        objective = {}
        for i, node in enumerate(candidate_nodes):
            cost = node.cost + self.distance(node, self.goal)
            objective[f'x{i}'] = cost
        qp.minimize(linear=objective)

        # Constraints: Select exactly one node
        linear_constraint = {f'x{i}': 1 for i in range(len(candidate_nodes))}
        qp.linear_constraint(linear=linear_constraint, sense='==', rhs=1)

        # Solve the problem using QAOA
        optimizer = COBYLA()  # Use a classical optimizer
        qaoa = QAOA(sampler=Sampler(), optimizer=optimizer, reps=1)  # Use fewer layers
        minimum_eigen_optimizer = MinimumEigenOptimizer(qaoa)
        result = minimum_eigen_optimizer.solve(qp)

        # Get the selected node
        selected_index = int(result.variable_names[0][1:])  # Extract index from variable name (e.g., 'x0' -> 0)
        return candidate_nodes[selected_index]

    def generate_path(self):
        for _ in range(self.max_iter):
            random_node = self.get_random_node()
            nearest_node = self.nearest_node(random_node)
            theta = math.atan2(random_node.y - nearest_node.y, random_node.x - nearest_node.x)
            new_node = Node(nearest_node.x + self.step_size * math.cos(theta), nearest_node.y + self.step_size * math.sin(theta))
            new_node.cost = nearest_node.cost + self.distance(nearest_node, new_node)

            if self.is_collision_free(nearest_node, new_node):
                candidate_nodes = [new_node] + self.nodes[:5]  # Use fewer candidate nodes
                best_node = self.quantum_optimization(candidate_nodes)
                best_node.parent = nearest_node
                self.nodes.append(best_node)

                # Disable dynamic drawing during execution
                self.dynamic_draw(best_node)

                if self.distance(best_node, self.goal) <= self.step_size:
                    self.goal.parent = best_node
                    self.nodes.append(self.goal)
                    return self.extract_path()
        return None

    def extract_path(self):
        path = []
        node = self.goal
        while node is not None:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

    def dynamic_draw(self, node):
        self.screen.fill((255, 255, 255))
        for ox, oy, size in self.obstacle_list:
            self.draw_circle(ox, oy, size, (255, 0, 0))  # Obstacles in red
        for n in self.nodes:
            if n.parent:
                self.draw_line(n.x, n.y, n.parent.x, n.parent.y, (0, 255, 0))  # Tree edges in green
        self.draw_circle(self.start.x, self.start.y, 5, (0, 0, 255))  # Start in blue
        self.draw_circle(self.goal.x, self.goal.y, 5, (255, 0, 0))  # Goal in red
        pygame.display.update()
        self.clock.tick(30)  # Control frame rate

    def draw_final_path(self, path):
        for i in range(len(path) - 1):
            self.draw_line(path[i][0], path[i][1], path[i + 1][0], path[i + 1][1], (0, 0, 255))  # Final path in blue
        pygame.display.update()

if __name__ == '__main__':
    start = (10, 10)
    goal = (300, 300)
    obstacle_list = [(100, 100, 20), (200, 300, 30), (350, 150, 40)]  # Obstacles as (x, y, radius)
    x_range = (0, 500)
    y_range = (0, 500)

    rrt_star = RRTStar(start, goal, obstacle_list, x_range, y_range)
    path = rrt_star.generate_path()

    if path:
        print("Path found:", path)
        rrt_star.draw_final_path(path)
        time.sleep(5)
    else:
        print("No path found")
    pygame.quit()

<ipython-input-13-4ad4ed5e02da>:72: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=optimizer, reps=1)  # Use fewer layers
<ipython-input-13-4ad4ed5e02da>:72: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=optimizer, reps=1)  # Use fewer layers
<ipython-input-13-4ad4ed5e02da>:72: DeprecationWarning: The class ``qiskit

Path found: [(10, 10), (28.13579897360436, 18.431654380310608), (41.02529779938928, 33.72416300447326), (60.83756660010702, 36.45802565064013), (60.53654612472848, 56.45576018916731), (78.45514224524652, 65.339673351156), (97.37661974296569, 71.8186925132906), (117.0543645418365, 68.24289055701385), (126.02191658450967, 86.11977536287273), (145.80513499401948, 89.05648599921949), (160.25643219932422, 102.88254867389124), (166.0453786537679, 122.02642756369787), (180.3444988057866, 136.009816383227), (171.27977711631516, 153.83762670004455), (163.19348689043997, 172.13002704507625), (154.43731672690836, 190.11139195620797), (166.84788977613928, 205.79506743709263), (179.67066941680213, 221.14356320335594), (188.4226886894015, 239.126948831542), (208.07270537726166, 235.40178518858386), (217.56925791326435, 253.0033615364835), (236.69659689190328, 258.8467256014926), (255.58808685253445, 252.28078263285727), (273.63175707469406, 243.65373418544775), (293.62753961908476, 244.0644414259975